## Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
import pandas as pd
import time
from time import sleep
from datetime import date

## ChromeDriver & Headers

In [2]:
options = Options()
options.add_experimental_option("detach", True)
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"}
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
time.sleep(1)

## Waiting Mechanism

In [3]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
wait = WebDriverWait(driver, 55)  # 55 seconds timeout

In [9]:
#Empty Lists to store product informations
dates = []
names = []
prices = []

## Define a function that will collect product informations by taking product ID

In [14]:
def p_check(code,model):
    driver.get('https://www.e-commerce.com') # Go to the website
    
    # Wait untill the page loads
    wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="ac search-toggler"]/i')))
    search = driver.find_element(By.XPATH, '//div[@class="ac search-toggler"]/i')
    search.click()
    input_field = driver.find_element(By.XPATH, '//input[@placeholder="Search"]')
    
    # Search the product by code
    input_field.send_keys(code)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@class="material-icons"]')))
    result = driver.find_element(By.XPATH, '//button[@class="material-icons"]')
    result.click()
    
    # Get the specfiq product from search result
    wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="p-item-details"]/h4/a')))
    url = driver.find_element(By.XPATH, '//div[@class="p-item-details"]/h4/a')
    url.click()
    
    # Match with the desired product
    p_model = driver.find_element(By.XPATH, '//*[@id="product"]/div/div[2]/ul/li[1]').text
    if p_model == model:
        # Check if the product is in stock
        try:
            check = driver.find_element(By.XPATH, '//div[@class="container__wrapper4"]/div').text
            if check == 'Out of Stock':
                prices.append('Out of Stock')
            else:
                #pass
        except:
            # Record current date
            dt = dates.append(date.today())
            # Product name
            name = driver.find_element(By.XPATH, '//h1[@class="product-name"]').text 
            names.append(name)
            # Product price
            price = driver.find_element(By.XPATH, '//*[@id="product"]/div/div[1]/table/tbody/tr[1]/td[2]/ins').text.rstrip('৳')
            prices.append(price)
    else:
        print('Product Unavailable')
        prices.append('NA') # In case desired product is unavailable

## Just give the product code and model number to execute the function

In [18]:
p_check(24216,'Model: Ryzen 7 7700X')# Product 1
p_check(34983,'Model: PRO MP225')#Product 2
p_check(16116,'Model: RTX 3060 VENTUS 2X OC')# Product 3

In [19]:
# Add the data to DataFrame
df = pd.DataFrame({'Date': dates, 'Name': names, 'Price': prices})
df

,Date,Name,Price
0,2024-06-08,AMD Ryzen 7 7700X Processor,"35,000"
1,2024-06-08,"MSI PRO MP225 21.5"" 100Hz IPS FHD Monitor","10,900"
2,2024-06-08,MSI GeForce RTX 3060 VENTUS 2X OC 12GB Graphic...,"38,800"


In [20]:
# Save as excel file
df.to_excel('Price_Tracking.xlsx', index=False)